<h1>This notebook is purely inspired from <a href="https://www.kaggle.com/code/smjishanulislam/quickstart-with-whisper-small/notebook"> here</a><h1>
   <h1>This notebook is created for utilizing GPU time. Because of limited GPU resources in Kaggle, training and inference in the same notebook is not a good idea. Use this notebook only for training. After training, import the model as a dataset. For inference, use the code in the <a href="https://www.kaggle.com/code/samratabduljalil/bengali-speech-recognition-inference/"> link</a>  <h1> 
       
       <h2>Must check GPU on or Off. best of luck </h2>

In [1]:
import os

import pandas as pd

import librosa
import librosa.display

import numpy as np

import IPython.display as ipd

import matplotlib.pyplot as plt

import random

from collections import Counter

from sklearn.model_selection import train_test_split

import torch
import torchaudio

from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import DatasetDict
from datasets import Dataset as DS

from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
    TrainingArguments,
    TrainerState,
    TrainerControl,
    EarlyStoppingCallback,
    pipeline
)

from torchmetrics.text import WordErrorRate, CharErrorRate

2024-04-24 04:56:52.360766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 04:56:52.360864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 04:56:52.483638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
BASE_DIR = '/kaggle/input/ben10/ben10'
train_data_dir = f"{BASE_DIR}/16_kHz_train_audio/"
test_data_dir = f"{BASE_DIR}/16_kHz_valid_audio/"
data_path = f"{BASE_DIR}/train.csv"

In [3]:
split2path = {
    "train": train_data_dir,
    "test": test_data_dir,
}

In [4]:
data = pd.read_csv(data_path)
data.sample(10)

,file_name,transcripts
8807,train_rangpur (256).wav,এই যে এখন তাইলে ইয়ে গুলে মিউচুয়েল করি দেই।\n\n...
1633,train_barishal (579).wav,তোমার দাদুর পোলা। দাদুর প্যাডেইড্ডা কি অইছে? অ...
11079,train_sylhet (1472).wav,অতো মামার লগে উঠিয়ার আমি। <> আমার। বন্দু <> রে...
10576,train_sylhet (969).wav,"অখানো ঘর বানাইয়া, অখন দেখা যায় তারা গাছ-বিরিক ..."
291,train_sandwip (292).wav,<> আমনে এক কেজি তরকারি বাজারে নিলে এখন আশি টাক...
4914,train_kishoreganj (644).wav,"আমিতো আরো ইডিট পারি না, কিচ্ছু পারি না। আমিওতো..."
873,train_sandwip (874).wav,সুদের পরে <> আমার টাকা লইয়েরে না আমার টাকা লই ...
4699,train_kishoreganj (429).wav,মার ধারো যাইত। এইম্মে আ । হেইযে তোর মা হেইযে আ...
8971,train_rangpur (420).wav,<> তোমরা <> ঘাস <> যাইয়া <> হয় নাই। <> কাইল ছা...
7141,train_narail (1202).wav,"হয়, বাড়ি আইছো? তা সে কি সে সরকারি চাকরি? ওর বা..."


In [5]:
def extract_split(filename):
    filename_ = filename.split("_")
    split = filename_[0]
    return split

def extract_district(filename):
    filename_ = filename.split(" ")[0]
    district = filename_.split("_")[1]
    return district

def beautify_dataset(data):
    splits = []
    districts = []
    newpaths = []
    transcripts = []
    
    for i in range(len(data)):
        filename, transcript = data.iloc[i]
        split = extract_split(filename)
        district = extract_district(filename)
        dir_path = split2path[split]
        composed_path = f"{dir_path}{filename}"
        
        if os.path.exists(composed_path) == False:
            print(f"{composed_path} does not exist.")
            continue
        
        # replace any newline characters
        transcript = transcript.replace("\n", " ")
        transcript = " ".join(transcript.split())
        
        splits.append(split)
        districts.append(district)
        newpaths.append(composed_path)
        transcripts.append(transcript)
    
    data['file_path'] = newpaths
    data['district'] = districts
    data['split'] = splits
    data['transcripts'] = transcripts
    
#     data.drop(columns=['file_name'], inplace=True)
    
    return data

In [6]:
data = beautify_dataset(data)
data.sample(20)

,file_name,transcripts,file_path,district,split
5408,train_kishoreganj (1138).wav,"না। পাচোয়াদ দে না? না। ও, আচ্ছা। এই মাঝে বন্ধ...",/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,kishoreganj,train
11733,train_sylhet (2126).wav,"কিতা? কিতা গো লাবিবা, বালা আছোনি? জি অয়, বালা ...",/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,sylhet,train
10048,train_sylhet (441).wav,হয়তো বা তারা যদি অখন আমারে দশটা টেখাও ভাড়া দেয়...,/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,sylhet,train
6471,train_narail (532).wav,তাও বাড়া তো উরা যে ফিলি। শেষে শেষে দিয়া লাগিছে...,/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,narail,train
6998,train_narail (1059).wav,সুমায় এওনো কতো সুমায় পইড়ে রইছে। তুমি আস্তে ধির...,/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,narail,train
10984,train_sylhet (1377).wav,"অখন বাদে বেটিয়ে খইছ্‌লো আকি। খয়, কিতা খইরো? অখ...",/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,sylhet,train
12342,train_sylhet (2735).wav,আরামে থাখার লাগি। তে অউ তারা তিন মাস বাদে এখ হ...,/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,sylhet,train
12400,train_sylhet (2793).wav,"কিলা, কিতা খরছিলাম বুলি গেছি, বাট কিছু ফুগামি-...",/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,sylhet,train
7364,train_narail (1425).wav,"ও বা এ <>। না পরে আসপানি, বিকেলে এক সুময়। নির্...",/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,narail,train
4296,train_kishoreganj (26).wav,"কুয়ন যা না, মানে জাস্ট খালি বিনোদন। যে কয়জন ...",/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,kishoreganj,train


In [7]:
data[data["transcripts"] == "<>"]
data[data["transcripts"] == ""]
data[data["transcripts"] == ".."]

,file_name,transcripts,file_path,district,split
1329,train_barishal (275).wav,..,/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,barishal,train
1331,train_barishal (277).wav,..,/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,barishal,train
1338,train_barishal (284).wav,..,/kaggle/input/ben10/ben10/16_kHz_train_audio/t...,barishal,train


In [8]:
# print(list(data[data['transcripts'] == ''].index))
data.drop(data[data['transcripts'] == ''].index, inplace=True)
      
# print(list(data[data['transcripts'] == '<>'].index))
data.drop(data[data['transcripts'] == "<>"].index, inplace=True)
      
# print(list(data[data['transcripts'] == '..'].index))
data.drop(data[data['transcripts'] == ".."].index, inplace=True)

In [9]:
data["transcripts"] = data["transcripts"].str.strip()

In [10]:
TASK = "transcribe"
MODEL_NAME = "openai/whisper-small"

In [11]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME, language='bn', task=TASK)
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language='bn', task=TASK)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
ids = tokenizer.encode("")
ids

[50258, 50302, 50359, 50363, 50257]

In [13]:
tokenizer.decode(ids)

'<|startoftranscript|><|bn|><|transcribe|><|notimestamps|><|endoftext|>'

In [14]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        
        torch.cuda.empty_cache()

        return batch

In [15]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [16]:
def prepare_dataset(example):
    audio_path = example["file_path"]
    
    # load the audio using librosa or torch audio (as you wish)
    audio, sr = librosa.load(audio_path, sr=16_000)
    
    example["input_features"] = feature_extractor(audio, sampling_rate=sr).input_features[0]
    
    example["labels"] = tokenizer(f"{example['transcripts']}", max_length=448, padding=True, truncation=True).input_ids
    
    return example


def filter_inputs(input_audio):
    """filter inputs with zero input length"""
    return 0 < len(input_audio)


def filter_labels(input_labels):
    """filter empty label sequences"""
    return 0 < len(input_labels)

In [17]:
train_df = data[data["split"] == "train"]

In [18]:
train_df=data[data["district"] == "kishoreganj"]

In [19]:
"""
    adjust test size accordingly.
"""
train_df, eval_df = train_test_split(train_df, test_size=0.01, shuffle=True)

In [20]:
len(train_df), len(eval_df)

(1637, 17)

In [21]:
ben_reg_voice_ds = DatasetDict()

train_split = DS.from_pandas(train_df)
eval_split = DS.from_pandas(eval_df)

ds_splits = DatasetDict({
    'train': train_split,
    'eval': eval_split
})

In [22]:
ds_splits = ds_splits.remove_columns(["split"])
print(ds_splits)

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcripts', 'file_path', 'district', '__index_level_0__'],
        num_rows: 1637
    })
    eval: Dataset({
        features: ['file_name', 'transcripts', 'file_path', 'district', '__index_level_0__'],
        num_rows: 17
    })
})


In [23]:
np.object = object

In [24]:
ds_splits = ds_splits.map(prepare_dataset, remove_columns=ds_splits.column_names["train"],
                          num_proc=2 # open for multithreadding
)

Map (num_proc=2):   0%|          | 0/1637 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/17 [00:00<?, ? examples/s]

In [25]:
len(ds_splits["train"]), len(ds_splits["eval"])

(1637, 17)

In [26]:
cer = CharErrorRate()
wer = WordErrorRate()

In [27]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer_res = wer(pred_str, label_str)
    cer_res = cer(pred_str, label_str)
    
    """
        uncomment the next 3 lines if you want to see how the examples look like during eval 
    """
    print("WER:",wer_res,"| CER:", cer_res) # to show up during running logs
    print("Pred:",pred_str[0])
    print("Label:",label_str[0])
    
    return {"wer": wer_res, "cer": cer_res}

In [28]:
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME, device_map="auto")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [29]:
model_id = "whisper-reg-ben" #you can use different model from hugging face 

In [30]:
#finetune this hyperparameter for getting the better result

training_args = Seq2SeqTrainingArguments(
    output_dir=model_id,
    per_device_train_batch_size=18,
    per_device_eval_batch_size=18,
    gradient_accumulation_steps=18,
    gradient_checkpointing=True,
    fp16=True,
    learning_rate=.0001,
    weight_decay=1e-2,
    warmup_steps=500,
    num_train_epochs=8,
    evaluation_strategy="steps", # or "epochs"
    predict_with_generate=True,
#     generation_max_length=448,
    save_steps=200,
    eval_steps=200,
    logging_steps=200,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    report_to="none",
    remove_unused_columns=False,
)

In [31]:
model.generation_config.language = "bn"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None
model.config.suppress_tokens = [] # added later

In [32]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds_splits["train"],
    eval_dataset=ds_splits["eval"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(2, 1.0)]
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [33]:
trainer.train()

# to use the high-level pipeline, ensure both the processor outputs and model outputs exist in the same dir
trainer.save_model(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


[]

<h1> go to output of the notebook you can find your best model .just create dataset just clicking on the three dot.then use that in inference notebook. inference notebook :<a href="">Click here</a></h1>